In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df_tasas_productos = pd.read_excel('./datos/tasas_productos.xlsx')
df_obligaciones_clientes = pd.read_excel('./datos/obligaciones_clientes.xlsx')

In [3]:
merged_df = df_obligaciones_clientes.merge(df_tasas_productos, 
                                           left_on=['cod_segm_tasa', 'cod_subsegm_tasa', 'cal_interna_tasa'], 
                                           right_on=['cod_segmento', 'cod_subsegmento', 'calificacion_riesgos'])


In [4]:
def obtener_tasa(row):
    if re.search(r'leasing', row['id_producto'], re.IGNORECASE):
        return row['tasa_leasing']
    elif re.search(r'cartera', row['id_producto'], re.IGNORECASE) and 'leasing' not in row['id_producto']:
        return row['tasa_cartera']
    elif re.search(r'operacion_especifica', row['id_producto'], re.IGNORECASE):
        return row['tasa_operacion_especifica']
    elif re.search(r'hipotecario', row['id_producto'], re.IGNORECASE):
        return row['tasa_hipotecario']
    elif re.search(r'sufi', row['id_producto'], re.IGNORECASE):
        return row['tasa_sufi']
    elif re.search(r'factoring', row['id_producto'], re.IGNORECASE):
        return row['tasa_factoring']
    elif re.search(r'tarjeta', row['id_producto'], re.IGNORECASE):
        return row['tasa_tarjeta']
    else:
        return 0

In [5]:

def obtener_valor_periodicidad(periodicidad):
    if periodicidad == 'MENSUAL':
        return 12
    elif periodicidad == 'BIMENSUAL':
        return 6
    elif periodicidad == 'TRIMESTRAL':
        return 4
    elif periodicidad == 'SEMESTRAL':
        return 2
    elif periodicidad == 'ANUAL':
        return 1
    else:
        return 1

In [6]:

    
def calcular_tasa_efectiva(row):
    return (1 + row['tasa'])**(1 / obtener_valor_periodicidad(row['periodicidad'])) - 1

In [7]:
def calcular_valor_final(row):
    return  row['tasa_efectiva'] * row['valor_inicial']

In [8]:
merged_df['tasa'] = merged_df.apply(obtener_tasa, axis=1)


merged_df['tasa_efectiva'] = merged_df.apply(calcular_tasa_efectiva, axis=1)
merged_df['valor_final'] = merged_df.apply(calcular_valor_final, axis=1)

final_df = merged_df[['radicado', 'num_documento', 'id_producto', 'valor_inicial', 'tasa', 'periodicidad', 'tasa_efectiva', 'valor_final']]

In [9]:
final_df

,radicado,num_documento,id_producto,valor_inicial,tasa,periodicidad,tasa_efectiva,valor_final
0,14427616502,1081648945,OEL - operacion_especifica,1.050000e+06,0.202839,MENSUAL,0.015509,16284.871534
1,14427616257,1032058622,0000000000097492172 - 29-Cartera Total,2.848500e+05,0.121857,MENSUAL,0.009628,2742.580271
2,14427615471,1081374249,OEL - leasing,7.528858e+04,0.134434,MENSUAL,0.010567,795.544732
3,14426546522,996136114,0000000090000272791 - 29-Cartera Total,6.240000e+05,0.193960,MENSUAL,0.014883,9286.758274
4,14427616197,1080662762,0000000090000242224 - 29-Cartera Total,1.909350e+05,0.190215,MENSUAL,0.014617,2790.889388
...,...,...,...,...,...,...,...,...
2043,14427616294,109325188,OPE - operacion_especifica,6.750000e+04,0.257292,SEMESTRAL,0.121291,8187.111077
2044,14427615890,960246199,RLP - cartera,5.325000e+05,0.279129,MENSUAL,0.020727,11037.037847
2045,14427615829,1080515925,OPE - operacion_especifica,6.586189e+04,0.209583,MENSUAL,0.015983,1052.650890
2046,14427616509,1080240535,OPE - operacion_especifica,6.000000e+04,0.210004,MENSUAL,0.016012,960.729324


In [10]:
df_sumas_clientes = merged_df.groupby('num_documento').agg(
    suma_valor_final=('valor_final', 'sum'),
    cantidad_productos=('id_producto', 'nunique')
)

df_sumas_clientes = df_sumas_clientes[df_sumas_clientes['cantidad_productos'] >= 2]

df_sumas_clientes.reset_index(inplace=True)

df_sumas_clientes

,num_documento,suma_valor_final,cantidad_productos
0,3907738,6741.584647,8
1,4234923,18035.643930,8
2,4935132,12880.579507,7
3,7061608,13730.266357,4
4,7099543,10100.998957,6
...,...,...,...
242,1339938146,3017.568190,2
243,1342616595,5636.059256,6
244,1344463827,11110.864288,4
245,1346212202,69440.932083,5
